In [1]:
import numpy as np
from scipy import spatial 
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

## Helper Function

In [2]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [3]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [4]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [5]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [6]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [9]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

FileNotFoundError: [Errno 2] No such file or directory: 'data/query_vectors.npy'

In [ ]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

In [ ]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

In [ ]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

In [ ]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed. 

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [ ]:
def generate_random_hash_functions(num_hashes, hash_size, dim):
    """
    Generates random hash functions for LSH.
    Args:
        num_hashes: Number of hash tables.
        hash_size: Size of each hash function (number of bits).
        dim: Dimensionality of the input vectors.
    Returns:
        List of random projection matrices.
    """
    return [np.random.randn(hash_size, dim) for _ in range(num_hashes)]


def hash_vector(vector, hash_function):
    """
    Hashes a vector using a given hash function.
    Args:
        vector: Input vector.
        hash_function: Random projection matrix.
    Returns:
        A tuple representing the hash key.
    """
    projection = np.dot(hash_function, vector)
    return tuple((projection > 0).astype(int))


In [ ]:
#TODO: Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.

def custom_indexing_algorithm(index_vectors, hash_functions):
    """
    Indexes vectors into hash tables using LSH.
    Args:
        index_vectors: Array of index vectors.
        hash_functions: List of random projection matrices.
    Returns:
        List of hash tables, each storing lists of vector indices.
    """
    hash_tables = [{} for _ in hash_functions]
    for i, vector in enumerate(index_vectors):
        for j, hash_function in enumerate(hash_functions):
            hash_key = hash_vector(vector, hash_function)
            if hash_key not in hash_tables[j]:
                hash_tables[j][hash_key] = []
            hash_tables[j][hash_key].append(i)
    return hash_tables

def custom_index_search(query_vectors, index_vectors, hash_functions, hash_tables, k, max_candidates=1000):
    """
    Searches for the k-nearest neighbors using LSH.
    Args:
        query_vectors: Array of query vectors.
        index_vectors: Array of index vectors.
        hash_functions: List of random projection matrices.
        hash_tables: List of hash tables.
        k: Number of nearest neighbors to retrieve.
        max_candidates: Number of candidate vectors considered from each hash table.
    Returns:
        Array of shape (n_queries, k) containing the indices of the k-nearest neighbors for each query vector.
    """
    result_indices = []
    for query_vec in query_vectors:
        candidate_indices = set()
        for hash_function, hash_table in zip(hash_functions, hash_tables):
            hash_key = hash_vector(query_vec, hash_function)
            if hash_key in hash_table:
                candidates = hash_table[hash_key]
                candidate_indices.update(candidates[:max_candidates])  # Limit the number of candidates
        if not candidate_indices:
            result_indices.append(np.array([]))
            continue
        candidate_indices = list(candidate_indices)
        candidate_vectors = index_vectors[candidate_indices]
        distances = np.linalg.norm(candidate_vectors - query_vec, axis=1)
        result_indices.append(np.array(candidate_indices)[np.argsort(distances)[:k]])
    return np.array(result_indices)
    

In [ ]:
# Add hyperparameters here (if needed)

num_hashes = 9
hash_size = int(dim**0.75)
hash_functions = generate_random_hash_functions(num_hashes, hash_size, dim)

In [ ]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, hash_functions)

In [ ]:
%%time
custom_index_ann = custom_index_search(query_vectors, index_vectors, hash_functions, custom_index, k, 800)

In [ ]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")